# Cartea Jaimungal Penalva 2015

In [ ]:
import sys
sys.path.append("../") # This version of the notebook is in the subfolder "notebooks" of the repo

import gym
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy

from copy import deepcopy


from mbt_gym.agents.BaselineAgents import *
from mbt_gym.gym.TradingEnvironment import TradingEnvironment
from mbt_gym.gym.helpers.generate_trajectory import generate_trajectory
from mbt_gym.gym.helpers.plotting import *
from mbt_gym.stochastic_processes.midprice_models import *
from mbt_gym.stochastic_processes.arrival_models import *
from mbt_gym.stochastic_processes.fill_probability_models import *
import torch
#print(torch.cuda.is_available())
#print(torch.cuda.get_device_name())

In [ ]:
from mbt_gym.gym.ModelDynamics import LimitOrderModelDynamics
from mbt_gym.rewards.RewardFunctions import CjMmCriterion
seed = 410
max_inventory = 100

In [ ]:
def get_env(num_trajectories:int = 1,
            initial_price = 100,
            terminal_time = 1.0,
            sigma = 2.0,
            n_steps = 1000,
            initial_inventory = 0,
            arrival_rate = 140,
            fill_exponent = 1.5,
            per_step_inventory_aversion = 0.01,
            terminal_inventory_aversion = 0.001):    
    midprice_model = BrownianMotionMidpriceModel(initial_price = initial_price, 
                                                 volatility=sigma, step_size=terminal_time/n_steps,
                                                 terminal_time = terminal_time,
                                                 num_trajectories=num_trajectories)
    arrival_model = PoissonArrivalModel(intensity=np.array([arrival_rate, arrival_rate]), 
                                        step_size=terminal_time/n_steps, 
                                        num_trajectories=num_trajectories)
    fill_probability_model = ExponentialFillFunction(fill_exponent=fill_exponent, 
                                                     step_size=terminal_time/n_steps,
                                                     num_trajectories=num_trajectories)
    LOtrader = LimitOrderModelDynamics(midprice_model = midprice_model, arrival_model = arrival_model, 
                                fill_probability_model = fill_probability_model,
                                num_trajectories = num_trajectories)
    reward = CjMmCriterion(per_step_inventory_aversion = per_step_inventory_aversion,
                           terminal_inventory_aversion = terminal_inventory_aversion,
                           terminal_time = terminal_time)
    env_params = dict(terminal_time=terminal_time, 
                      n_steps=n_steps,
                      seed = seed,
                      initial_inventory = initial_inventory,
                      model_dynamics = LOtrader,
                      reward_function = reward,
                      max_inventory=max_inventory,
                      normalise_action_space = False,
                      normalise_observation_space = False,
                      num_trajectories=num_trajectories)
    return TradingEnvironment(**env_params)

In [ ]:
env = get_env()
agent = CarteaJaimungalMmAgent(env = env, max_inventory = max_inventory)

In [ ]:
plot_trajectory(env, agent, seed = seed)

### Comparing the value function to the simulated optimal agent 

In [ ]:
num_trajectories = 1_000
vec_env = get_env(num_trajectories)
vec_agent = CarteaJaimungalMmAgent(env = vec_env, max_inventory = max_inventory)

In [ ]:
observations, actions, rewards = generate_trajectory(vec_env, vec_agent)

In [ ]:
results, fig, total_rewards = generate_results_table_and_hist(vec_env=vec_env,agent=vec_agent)

# Value function versus total rewards

In [ ]:
vec_env.reset()
agent = CarteaJaimungalMmAgent(env = vec_env, max_inventory = max_inventory)
agent.calculate_true_value_function(vec_env.state[0].reshape(1,-1))

In [ ]:
np.mean(total_rewards), np.std(total_rewards)

In [ ]:
true_mean = agent.calculate_true_value_function(vec_env.state[0].reshape(1,-1))[0,0]
sample_mean = np.mean(total_rewards)
N = len(total_rewards)
sample_variance = np.var(total_rewards) * N/(N-1)
T = (sample_mean -  true_mean)/ (np.sqrt(sample_variance) / np.sqrt(N))
q_l, q_u = scipy.stats.t(df=(N-1)).ppf((0.1, 0.9))
if T>q_l and T<q_u:
    print('We do not have evidence to reject the hypothesis that the means are not the same')
else:
    print('We have evidence to reject the hypothesis that the means are the same')

# Alternative model parameters -- Part I

In [ ]:
num_trajectories = 1_000
vec_env = get_env(num_trajectories, initial_price=150,
                    terminal_time=1.0,
                    sigma=1.0,
                    n_steps=1000,
                    initial_inventory=0,
                    arrival_rate=100,
                    fill_exponent=1.0)
vec_agent = CarteaJaimungalMmAgent(env = vec_env, max_inventory = max_inventory)
observations, actions, rewards = generate_trajectory(vec_env, vec_agent)
results, fig, total_rewards = generate_results_table_and_hist(vec_env=vec_env,agent=vec_agent)

In [ ]:
vec_env.reset()
agent = CarteaJaimungalMmAgent(env = vec_env, max_inventory = max_inventory)
agent.calculate_true_value_function(vec_env.state[0].reshape(1,-1))

In [ ]:
np.mean(total_rewards), np.std(total_rewards)

In [ ]:
true_mean = agent.calculate_true_value_function(vec_env.state[0].reshape(1,-1))[0,0]
sample_mean = np.mean(total_rewards)
N = len(total_rewards)
sample_variance = np.var(total_rewards) * N/(N-1)
T = (sample_mean -  true_mean)/ (np.sqrt(sample_variance) / np.sqrt(N))
q_l, q_u = scipy.stats.t(df=(N-1)).ppf((0.1, 0.9))
if T>q_l and T<q_u:
    print('We do not have evidence to reject the hypothesis that the means are the same')
else:
    print('We have evidence to reject the hypothesis that the means are the same')

# Alternative model parameters -- Part II

In [ ]:
num_trajectories = 1_000
vec_env = get_env(num_trajectories, initial_price=50,
                    terminal_time=1.0,
                    sigma=1.5,
                    n_steps=2000,
                    initial_inventory=0,
                    arrival_rate=50,
                    fill_exponent=2.0)
vec_agent = CarteaJaimungalMmAgent(env = vec_env, max_inventory = max_inventory)
observations, actions, rewards = generate_trajectory(vec_env, vec_agent)
results, fig, total_rewards = generate_results_table_and_hist(vec_env=vec_env,agent=vec_agent)

In [ ]:
vec_env.reset()
agent = CarteaJaimungalMmAgent(env = vec_env, max_inventory = max_inventory)
agent.calculate_true_value_function(vec_env.state[0].reshape(1,-1))

In [ ]:
np.mean(total_rewards), np.std(total_rewards)

In [ ]:
true_mean = agent.calculate_true_value_function(vec_env.state[0].reshape(1,-1))[0,0]
sample_mean = np.mean(total_rewards)
N = len(total_rewards)
sample_variance = np.var(total_rewards) * N/(N-1)
T = (sample_mean -  true_mean)/ (np.sqrt(sample_variance) / np.sqrt(N))
q_l, q_u = scipy.stats.t(df=(N-1)).ppf((0.1, 0.9))
if T>q_l and T<q_u:
    print('We do not have evidence to reject the hypothesis that the means are the same')
else:
    print('We have evidence to reject the hypothesis that the means are the same')

# Alternative model parameters -- Part III

In [ ]:
num_trajectories = 1_000
vec_env = get_env(num_trajectories, initial_price=50,
                    terminal_time=2.0,
                    sigma=1.5,
                    n_steps=2000,
                    initial_inventory=0,
                    arrival_rate=50,
                    fill_exponent=2.0)
vec_agent = CarteaJaimungalMmAgent(env = vec_env, max_inventory = max_inventory)
#observations, actions, rewards = generate_trajectory(vec_env, vec_agent)
results, fig, total_rewards = generate_results_table_and_hist(vec_env=vec_env,agent=vec_agent, n_episodes=num_trajectories)

In [ ]:
vec_env.reset()
agent = CarteaJaimungalMmAgent(env = vec_env, max_inventory = max_inventory)
agent.calculate_true_value_function(vec_env.state[0].reshape(1,-1))

In [ ]:
np.mean(total_rewards), np.std(total_rewards)

In [ ]:
true_mean = agent.calculate_true_value_function(vec_env.state[0].reshape(1,-1))[0,0]
sample_mean = np.mean(total_rewards)
N = len(total_rewards)
sample_variance = np.var(total_rewards) * N/(N-1)
T = (sample_mean -  true_mean)/ (np.sqrt(sample_variance) / np.sqrt(N))
q_l, q_u = scipy.stats.t(df=(N-1)).ppf((0.1, 0.9))
if T>q_l and T<q_u:
    print('We do not have evidence to reject the hypothesis that the means are the same')
else:
    print('We have evidence to reject the hypothesis that the means are the same')